# Face to Face Conversation System using Deep Neural Networks

The project involves the following steps taken by Rahul Sharma with reference from the research paper.<br>

1. Create a dummy dataset(since collection of original dataset will take a lot of time)<br>
2. Create an encoder decoder architecture for the speaking and listening model
3. Implement the conversational model
4. Test the speaking and listening models
5. Take the output of the speaking and listening models
6. Implement Pix2Pix HD GAN to get the final output

In the middle there will be certain tests to perform which will come as the project proceeds.

## 1. Creation of a dummy dataset

The dummy dataset needs the Facial Action Units + Face Pose of the listener and speaker. These are 20 dimensional vectors which are created arbitrarily from the OpenFace software outputs.

!CAUTION!: the data used in this notebook is arbitrarily generated. The final data that will be used will come from read videos of conversations

In [28]:
#importing the reqyired modules
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import random

pathToData = "./videoplayback.csv"
rawData    = pd.read_csv(pathToData)
columns    = list(rawData.columns)
rawData

,frame,face_id,timestamp,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,-0.008,0.413,0.112,1.55,0.00,0.41,0.0,...,0,0,0,1,1,1,0,0,0,1
1,2,0,0.033,-0.018,0.399,0.124,1.58,0.00,0.36,0.0,...,0,0,0,1,1,1,0,0,0,1
2,3,0,0.067,-0.020,0.397,0.133,1.55,0.00,0.34,0.0,...,0,0,0,1,1,1,0,0,0,1
3,4,0,0.100,-0.026,0.396,0.144,1.42,0.00,0.37,0.0,...,0,0,0,1,1,1,0,0,0,1
4,5,0,0.133,-0.031,0.397,0.155,1.24,0.00,0.39,0.0,...,0,0,0,1,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,2432,0,81.114,-0.206,-0.859,-0.321,0.87,0.74,0.21,0.0,...,0,1,1,0,0,1,0,0,0,1
2432,2433,0,81.148,-0.191,-0.770,-0.282,0.67,0.45,0.00,0.0,...,1,1,1,0,0,1,0,0,0,1
2433,2434,0,81.181,-0.125,-0.727,-0.246,0.54,0.23,0.00,0.0,...,1,1,1,0,0,1,0,0,0,1
2434,2435,0,81.214,-0.068,-0.708,-0.214,0.50,0.00,0.00,0.0,...,1,1,1,0,0,1,0,0,0,0


In [12]:
#taking the columns where we have the Pose and 17 action units (regression)
filterColumns = columns[3:]
filterColumns = [i for i in filterColumns if "_r" in i or "pose" in i]
filterData    = rawData[filterColumns]
filterData

,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,-0.008,0.413,0.112,1.55,0.00,0.41,0.0,0.08,0.00,0.56,0.19,0.00,0.35,0.00,2.11,0.44,0.00,0.00,0.00,0.74
1,-0.018,0.399,0.124,1.58,0.00,0.36,0.0,0.03,0.00,0.55,0.19,0.00,0.36,0.00,2.03,0.41,0.00,0.00,0.00,0.57
2,-0.020,0.397,0.133,1.55,0.00,0.34,0.0,0.01,0.00,0.52,0.22,0.00,0.36,0.00,1.99,0.33,0.07,0.00,0.00,0.56
3,-0.026,0.396,0.144,1.42,0.00,0.37,0.0,0.00,0.00,0.53,0.20,0.00,0.28,0.00,2.05,0.30,0.16,0.00,0.00,0.50
4,-0.031,0.397,0.155,1.24,0.00,0.39,0.0,0.00,0.00,0.54,0.20,0.00,0.30,0.00,2.05,0.23,0.30,0.00,0.00,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,-0.206,-0.859,-0.321,0.87,0.74,0.21,0.0,0.06,0.98,0.00,0.95,0.57,0.76,0.00,0.64,0.00,0.04,0.67,1.65,1.48
2432,-0.191,-0.770,-0.282,0.67,0.45,0.00,0.0,0.18,1.26,0.00,1.25,0.76,1.06,0.23,0.47,0.00,0.04,0.08,1.23,1.48
2433,-0.125,-0.727,-0.246,0.54,0.23,0.00,0.0,0.31,1.76,0.00,1.26,0.99,1.49,0.85,0.43,0.00,0.04,0.00,0.70,1.36
2434,-0.068,-0.708,-0.214,0.50,0.00,0.00,0.0,0.43,2.05,0.00,1.16,1.03,1.84,1.39,0.34,0.00,0.00,0.00,0.22,1.10


The filter columns now have the 20 dimensional vector that is needed to go into the encoder decoder architecture. Now, let's create the listener's dummy dataset for X,Y pairs of data points for our Enc-Dec speaking/listening model.

In [20]:
#let's see some information about the data
filterData.describe()

,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
count,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.00000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000
mean,0.100613,-0.183204,-0.066628,0.503112,0.286720,0.446786,0.069557,0.434269,0.583543,0.143580,1.020205,0.514840,0.449503,0.973247,0.94624,0.165238,0.328354,0.798112,0.680447,0.181880
std,0.138948,0.418088,0.148935,0.776032,0.672829,0.688002,0.165909,0.422902,0.641360,0.284122,0.693222,0.720615,0.581334,1.399184,0.79578,0.316957,0.599710,0.711267,0.646511,0.305462
min,-0.225000,-0.906000,-0.537000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.022500,-0.407000,-0.133000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.480000,0.000000,0.000000,0.000000,0.22000,0.000000,0.000000,0.110000,0.090000,0.000000
50%,0.097000,-0.305000,-0.078000,0.000000,0.000000,0.090000,0.000000,0.360000,0.340000,0.000000,0.930000,0.130000,0.190000,0.120000,0.73000,0.000000,0.000000,0.660000,0.515000,0.000000
75%,0.188000,0.085250,0.022000,0.980000,0.090000,0.580000,0.000000,0.740000,1.030000,0.120000,1.502500,0.740000,0.692500,1.632500,1.84000,0.230000,0.440000,1.370000,1.170000,0.300000
max,1.216000,0.560000,0.188000,4.950000,4.980000,3.310000,1.400000,2.290000,3.000000,2.170000,3.260000,3.050000,2.810000,5.000000,2.87000,2.480000,2.250000,3.720000,2.960000,1.870000


Based on the description of the data, we will take the min and max values of each column. The values for the dummy dataset will be produced arbitrarily from the range of each column. This means that the dataset above will be the input to the enc-dec model and the dataset that we are generating in this section will be the output. Note that both input and output datasets for the listening model are in 20 dimensions.

In [36]:
n = 2436
dummyData = pd.DataFrame(columns = filterColumns)
for colname in filterColumns:
    maximum = filterData[colname].max()
    minimum = filterData[colname].min()
    valueList = []

    for i in range(2436):
        randomValue = random.uniform(minimum, maximum)
        valueList.append(randomValue)
    
    dummyData[colname] = valueList


In [39]:
#saving the dummyData
pd.DataFrame.to_csv(dummyData, "dummyData.csv")